In [ ]:
import logging
from logging.handlers import RotatingFileHandler

# Set up logger
logger = logging.getLogger('data_ingestion')
logger.setLevel(logging.DEBUG)

# Console handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# File handler with rotation
file_handler = RotatingFileHandler('data_ingestion.log', maxBytes=5*1024*1024, backupCount=3)
file_handler.setLevel(logging.DEBUG)

# Log message format
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
file_handler.setFormatter(formatter)

# Add handlers to the logger
logger.addHandler(console_handler)
logger.addHandler(file_handler)

# Usage in data ingestion
def ingest_data(file_path):
    try:
        logger.info(f"Starting data ingestion for file: {file_path}")
        # Simulate data ingestion
        if not file_path.endswith('.csv'):
            raise ValueError("Unsupported file format")
        logger.info(f"Successfully ingested file: {file_path}")
    except Exception as e:
        logger.error(f"Error during data ingestion: {e}", exc_info=True)

# Example usage
ingest_data('example.csv')
ingest_data('example.txt')  # This will log an error


Connected to Salesforce Prod


In [2]:
claims_sf = updated_data_pull(['Claims__c'], sf_prod)
claims_sf.shape

: 

: 

: 

In [3]:
import os
os.chdir('/mnt/c/Users/ralvin/OneDrive - Reliant Health Partners/Documents/RHP_dev_RA/Automate_skyvia')
from helper_functions_v2 import create_df, updated_data_pull, upsert, delete_record, insert_records, salesforce_connection
import pandas as pd
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 


sf = salesforce_connection(sandbox=False)

object_names = ['Account']




store = []
for i in object_names:
    sf_object = getattr(sf, i)

    # Retrieve and print field names
    metadata = sf_object.describe()
    field_names = [field['name'] for field in metadata['fields']]
    print(f"Field names for {i}: {field_names}")

    # Retrieve and print record data
    query = f"SELECT {', '.join(field_names)} FROM {i}"  # Fetch all fields
    records = sf.query(query)
    all_records = records['records']

    # Handle pagination
    while not records['done']:
        records = sf.query_more(records['nextRecordsUrl'], True)
        all_records.extend(records['records'])

    # Convert to DataFrame
    df = pd.DataFrame(all_records)



    
   


Connected to Salesforce Prod
Field names for Account: ['Id', 'IsDeleted', 'MasterRecordId', 'Name', 'Type', 'RecordTypeId', 'ParentId', 'BillingStreet', 'BillingCity', 'BillingState', 'BillingPostalCode', 'BillingCountry', 'BillingStateCode', 'BillingCountryCode', 'BillingLatitude', 'BillingLongitude', 'BillingGeocodeAccuracy', 'BillingAddress', 'ShippingStreet', 'ShippingCity', 'ShippingState', 'ShippingPostalCode', 'ShippingCountry', 'ShippingStateCode', 'ShippingCountryCode', 'ShippingLatitude', 'ShippingLongitude', 'ShippingGeocodeAccuracy', 'ShippingAddress', 'Phone', 'Fax', 'Website', 'PhotoUrl', 'Industry', 'AnnualRevenue', 'NumberOfEmployees', 'TickerSymbol', 'Description', 'OwnerId', 'CreatedDate', 'CreatedById', 'LastModifiedDate', 'LastModifiedById', 'SystemModstamp', 'LastActivityDate', 'LastViewedDate', 'LastReferencedDate', 'IsPartner', 'IsCustomerPortal', 'Jigsaw', 'JigsawCompanyId', 'AccountSource', 'SicDesc', 'Copy_Billing_Address_to_Shipping_Address__c', 'rh2__testCur

In [4]:
df.shape

(53339, 98)

In [21]:
from typing import List


sf = salesforce_connection(sandbox=True)

def updated_data_pull(object_names: List[str], sf):
    store = {}
    for i in object_names:
        sf_object = getattr(sf, i)

        # Retrieve and print field names
        metadata = sf_object.describe()
        field_names = [field['name'] for field in metadata['fields']]
        # print(f"Field names for {i}: {field_names}")

        # Retrieve and print record data
        query = f"SELECT {', '.join(field_names)} FROM {i}"  # Fetch all fields
        records = sf.query(query)
        all_records = records['records']

        # Handle pagination
        while not records['done']:
            records = sf.query_more(records['nextRecordsUrl'], True)
            all_records.extend(records['records'])

        # Convert to DataFrame
        df = pd.DataFrame(all_records)

        store[i] = df
        print(f'pulled {i}')

    if len(store) == 1:
        return store[i]
    else:
        return store



store = updated_data_pull(['Account'], sf)


Connected to Salesforce sandbox
pulled Account


In [ ]:


Source_data = pd.read_csv("/mnt/c/Users/ralvin/Downloads/837I00111182024.csv")


print(Source_data.shape)
# Source_data.head(10)

(446, 42)


In [35]:
duplicates = Source_data[Source_data.duplicated(subset=['Claim ID'], keep=False)]

duplicates

,Claim ID,Claim Number,Line ID,TIN,Provider,Address,City,State,Zip,Patient ID,Patient,Date of Birth,Patient Group/Policy Number,DRG,Diag Code 1st,Diag Code 2nd,Diag Code 3rd,Diag Code 4th,Begin DOS,End DOS,Revenue Code,HCPCS/CPT Code,Mod,Mod (2nd),Units,Billed Amount,MAR,IsQPA,Billing Provider Taxonomy,JurisdictionState,4LCodes,NPI,PlaceofService,AnesthesiaUnits,Billtype,BillProvAddress1,BillProvAddress2,BillProvCity,BillProvState,BillProvZip,DRGCode,ClientName
2,317D671330C4442tksft,U424319281216,1,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,3/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,278,C1889,NaN,NaN,1,3.00,3.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP
3,317D671330C4442tksft,U424319281216,2,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,3/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,300,81025,NaN,NaN,1,54.00,54.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP
4,317D671330C4442tksft,U424319281216,3,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,3/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,310,88305,59,NaN,4,1000.00,1000.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP
5,317D671330C4442tksft,U424319281216,4,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,3/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,310,88342,NaN,NaN,1,1159.00,1159.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP
6,317D671330C4442tksft,U424319281216,5,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,3/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,370,NaN,NaN,NaN,3,1227.00,1227.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP
7,317D671330C4442tksft,U424319281216,6,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,3/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,636,J2704,NaN,NaN,70,155.00,155.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP
8,317D671330C4442tksft,U424319281216,7,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,3/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,710,NaN,NaN,NaN,3,48.00,48.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP
9,317D671330C4442tksft,U424319281216,8,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,3/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,750,43239,NaN,NaN,1,3326.00,3326.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP
10,317D671330C4442tksft,U424319281216,9,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,3/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,750,45385,PT,NaN,1,3326.00,3326.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP
11,B7D4248A586D86Ftksft,E24317I1000001,1,943377073,MARIN SPECIALTY SURGERY CENTER,1 THORNDALE DR STE 210,SAN RAFAEL,CA,949030070,142729742,REUSSWIG ANNE-LISE,2/26/1971,241160,NaN,M4802,M5412,M2578,G992,3/13/2024,3/13/2024,490,22551,NaN,NaN,1,55585.00,55585.00,N,49 - NOT CROSS REFERENCED,CA,C000,NaN,24,,24,1 THORNDALE DR STE 210,,SAN RAFAEL,CA,949030070,,IAA - Reliant


In [2]:

##### Data key, mapping source is on skyvia
# 3 tables to populate on SF is the end goal.... update we want to update accoutns table to 
# Provider_TIN__c
# Claims__c
# Line_Items__c




Provider_TIN__c = ['TIN', 'Address', 'City', 'State', 'Zip']
Claims__c = ['JurisdictionState', 'TIN', 'Diag Code 1st', 'Diag Code 2nd', 'Diag Code 3rd', 'Diag Code 4th', 'Provider', 'Patient Group/Policy Number', 'Claim ID',
              'Patient', 'Patient ID', 'Zip', 'DRG', 'IsQPA', 'NPI']
Line_Items__c = ['Begin DOS', 'Revenue Code', 'Billed Amount', 'Claim ID', 'Mod', 'Units', 'MAR', 'HCPCS/CPT Code', 'Line ID']



###FMP and Flagged not found in source but in skyvia
### fields on source not on skyvia mapping:
# ['DRGCode',
#  'BillProvZip',
#  'Date of Birth',
#  'Claim Number',
#  'End DOS',
#  'Billing Provider Taxonomy',
#  '4LCodes',
#  'BillProvAddress1',
#  'PlaceofService',
#  'ClientName',
#  'BillProvCity',
#  'Mod (2nd)',
#  'Billtype',
#  'BillProvState',
#  'AnesthesiaUnits',
#  'BillProvAddress2']



# print(claims_data.columns)
# print(Provider_TIN__c + Claims__c + Line_Items__c)



Source_data.loc[:10, Provider_TIN__c]

,TIN,Address,City,State,Zip
0,941105628,FILE 54602,LOS ANGELES,CA,900749998
1,351166081,1125 W JEFFERSON STREET,FRANKLIN,IN,461312140
2,610461753,PO BOX 638704,CINCINNATI,OH,452638704
3,610461753,PO BOX 638704,CINCINNATI,OH,452638704
4,610461753,PO BOX 638704,CINCINNATI,OH,452638704
5,610461753,PO BOX 638704,CINCINNATI,OH,452638704
6,610461753,PO BOX 638704,CINCINNATI,OH,452638704
7,610461753,PO BOX 638704,CINCINNATI,OH,452638704
8,610461753,PO BOX 638704,CINCINNATI,OH,452638704
9,610461753,PO BOX 638704,CINCINNATI,OH,452638704


In [2]:
from simple_salesforce import Salesforce, SalesforceLogin
import pyodbc, sys, time 
import requests




def salesforce_connection(sandbox=True):
    session = requests.Session()
    security_token = 'pIjwc8P0OB2ZisbQYhlKxfLd'
    username = 'ralvin@relianthp.com.tctdev'
    password = 'Flash363!'
    domain = 'test'

    session_id, instance = SalesforceLogin(username=username, password=password, security_token=security_token, domain=domain)
    sf = Salesforce(instance=instance, session_id=session_id, session=session)
    print('Connected to Salesforce (Sandbox)')
    return sf

sf = salesforce_connection()





Connected to Salesforce (Sandbox)


In [3]:
all_objects = sf.describe()["sobjects"]


claims_metadata = sf.Claims__c.describe()
claim_cols = [field['name'] for field in claims_metadata['fields']]
# print(claim_cols)
# print(len(claim_cols))
line_item_metadata = sf.Line_Items__c.describe()
line_item_cols = [field['name'] for field in line_item_metadata['fields']]
# print(line_item_cols)
# print(len(line_item_cols))
provider_tin_metadata = sf.Provider_TIN__c.describe()
provider_tin_cols = [field['name'] for field in provider_tin_metadata['fields']]
# print(provider_tin_col)
# print(len(provider_tin_col))
jurisdiction_metadata = sf.Jurisdiction__c.describe()
jurisdiction_cols = [field['name'] for field in jurisdiction_metadata['fields']]
# print(jurisdiction_col)
# print(len(sorted(jurisdiction_col)))
group_clients_metadata = sf.Groups_Clients__c.describe()
group_clients_cols = [field['name'] for field in group_clients_metadata['fields']]
# print(group_clients_cols)
# print(len(sorted(group_clients_cols)))
drg_metadata = sf.DRG__c.describe()
drg_cols = [field['name'] for field in drg_metadata['fields']]
# print(drg_cols)
# print(len(sorted(drg_cols)))
hcpcs_cpt_metadata = sf.HCPCS_CPT_Code__c.describe()
hcpcs_cpt_cols = [field['name'] for field in hcpcs_cpt_metadata['fields']]
# print(hcpcs_cpt_cols)
# print(len(sorted(hcpcs_cpt_cols)))
accounts_metadata = sf.Account.describe()
account_cols = [field['name'] for field in accounts_metadata['fields']]

needed_objects = { "Claims__c": claim_cols, "Line_Items__c": line_item_cols, "Provider_TIN__c": provider_tin_cols, 
                 'Jurisdiction__c': jurisdiction_cols, 'Groups_Clients__c': group_clients_cols, 'DRG__c': drg_cols, 
                 'HCPCS_CPT_Code__c': hcpcs_cpt_cols, 'Account':account_cols}


for table in needed_objects:
    print(table)
    print(needed_objects[table])
    print(len(needed_objects[table]))

Claims__c
['Id', 'IsDeleted', 'Name', 'CreatedDate', 'CreatedById', 'LastModifiedDate', 'LastModifiedById', 'SystemModstamp', 'LastActivityDate', 'LastViewedDate', 'LastReferencedDate', 'Duplicate__c', 'Client__c', 'Claim_Number__c', 'Jurisdiction__c', 'Provider_TIN__c', 'Contains_Line_Items__c', 'Status__c', 'Appeal__c', 'Date_Fee_Paid__c', 'Fee_Credit_Date__c', 'Diag_Code__c', 'Diag_Code_2nd__c', 'Diag_Code_3rd__c', 'Proc_Code__c', 'Proc_Code_2nd__c', 'Diag_Code_4th__c', 'Total_Billed__c', 'Provider__c', 'Group_Client__c', 'Claim_ID__c', 'Received_Date__c', 'Appeal_Rec_d_Date__c', 'Last_Contact__c', 'Appeal_Stage__c', 'State_Appeal__c', 'Contact__c', 'Phone__c', 'Fax__c', 'Email__c', 'Notes__c', 'Appeal_Closed_Date__c', 'POS__c', 'Attorney_Assigned__c', 'Detailed_Appeal_Resp__c', 'Begin_DOS__c', 'End_DOS__c', 'Total_MAR__c', 'Total_Allowed__c', 'Total_Reductions__c', 'Fee__c', 'Patient__c', 'Date_of_Birth__c', 'Patient_ID__c', 'Total_Additional_Payment__c', 'Fee_Credit__c', 'Savings_

In [13]:

table_store = {}

for object_name in needed_objects:
    print(object_name)

    object_fields = needed_objects[object_name]

    fields_str = ", ".join(object_fields)
    query = f"SELECT {fields_str} FROM {object_name}"
    results = sf.query_all(query)
    records = results['records']
    df = pd.DataFrame(records).drop(columns='attributes')  # Remove unwanted attributes column
    table_store[object_name] = df


Claims__c
Line_Items__c
Provider_TIN__c
Jurisdiction__c
Groups_Clients__c
DRG__c
HCPCS_CPT_Code__c
Account


In [5]:
for key in table_store:    ############ pulled data tables to make updates on 
    print(key)
    print(table_store[key].shape)
    



Claims__c
(12919, 99)
Line_Items__c
(22745, 47)
Provider_TIN__c
(10823, 27)
Jurisdiction__c
(53, 16)
Groups_Clients__c
(1702, 58)
DRG__c
(808, 20)
HCPCS_CPT_Code__c
(10045, 36)
Account
(2217, 98)


In [26]:
table_store['Jurisdiction__c'].head()

,Id,OwnerId,IsDeleted,Name,CreatedDate,CreatedById,LastModifiedDate,LastModifiedById,SystemModstamp,LastActivityDate,LastViewedDate,LastReferencedDate,WC_Language__c,FS_Link__c,Appeal_Timeframe__c,Appeal_Link__c
0,a024W00000HYAj6QAH,0054W00000ASrc5QAD,False,AK,2020-09-03T18:38:20.000+0000,0054W00000ASrc5QAD,2020-09-03T18:38:20.000+0000,0054W00000ASrc5QAD,2020-09-03T18:38:20.000+0000,None,None,None,None,labor.state.ak.us/wc/home.htm,30 Days From Receipt Of Medical Report,None
1,a024W00000HYAj7QAH,0054W00000ASrc5QAD,False,AL,2020-09-03T18:38:20.000+0000,0054W00000ASrc5QAD,2020-09-03T18:38:20.000+0000,0054W00000ASrc5QAD,2020-09-03T18:38:20.000+0000,None,None,None,Alabama work comp guidelines,https://labor.alabama.gov/wc/feeschedules.aspx,None,https://labor.alabama.gov/wc/medical.aspx
2,a024W00000HYAj8QAH,0054W00000ASrc5QAD,False,AR,2020-09-03T18:38:20.000+0000,0054W00000ASrc5QAD,2020-09-03T18:38:20.000+0000,0054W00000ASrc5QAD,2020-09-03T18:38:20.000+0000,None,None,None,Arkansas Workers' Comp guidelines,www.awcc.state.ar.us/medfeetoc.html,30 days or 60 says if no response,http://www.awcc.state.ar.us/rules/rule30.pdf
3,a024W00000HYAj9QAH,0054W00000ASrc5QAD,False,AZ,2020-09-03T18:38:20.000+0000,0054W00000ASrc5QAD,2021-03-04T19:09:31.000+0000,0054W00000ASrc5QAD,2021-03-04T19:09:31.000+0000,None,None,None,Arizona work comp guidelines,http://www.ica.state.az.us/Claims/Claims_main....,No State appeals,http://www.ica.state.az.us/Claims/Claims_main....
4,a024W00000HYAjAQAX,0054W00000ASrc5QAD,False,CO,2020-09-03T18:38:20.000+0000,0054W00000ASrc5QAD,2023-07-10T16:18:31.000+0000,0054W00000ASrc5QAD,2023-07-10T16:18:31.000+0000,None,None,None,Colorado work comp guidelines and fee schedule,https://www.colorado.gov/pacific/sites/default...,60 Days,https://www.colorado.gov/pacific/cdle/statute-...


In [ ]:
###### DATA DICTIONARY:


Provider_TIN_dict = {'TIN':['Name', 'Unique_TIN__c'], 'Address':['Address__c'], 'City':['City__c'], 'State':['State__c'], 'Zip':['Zip__c']}



Claims_dict = ['JurisdictionState', 'TIN', 'Diag Code 1st', 'Diag Code 2nd', 'Diag Code 3rd', 'Diag Code 4th', 'Provider', 'Patient Group/Policy Number', 'Claim ID',
              'Patient', 'Patient ID', 'Zip', 'DRG', 'IsQPA', 'NPI']
# Line_Items__c = ['Begin DOS', 'Revenue Code', 'Billed Amount', 'Claim ID', 'Mod', 'Units', 'MAR', 'HCPCS/CPT Code', 'Line ID']


Source_data.head(2)

,Claim ID,Claim Number,Line ID,TIN,Provider,Address,City,State,Zip,Patient ID,Patient,Date of Birth,Patient Group/Policy Number,DRG,Diag Code 1st,Diag Code 2nd,Diag Code 3rd,Diag Code 4th,Begin DOS,End DOS,Revenue Code,HCPCS/CPT Code,Mod,Mod (2nd),Units,Billed Amount,MAR,IsQPA,Billing Provider Taxonomy,JurisdictionState,4LCodes,NPI,PlaceofService,AnesthesiaUnits,Billtype,BillProvAddress1,BillProvAddress2,BillProvCity,BillProvState,BillProvZip,DRGCode,ClientName
0,A3038B5E4B6C914tksft,4.32E+14,1,941105628,KAISER FOUNDATION HOSPITALS,FILE 54602,LOS ANGELES,CA,900749998,569713835,WATKINS MONIC,8/12/1982,NNMCMC,NaN,M25562,NaN,NaN,NaN,9/26/2024,9/26/2024,320,73564,NaN,NaN,1,987.0,156.72,N,282N00000X,CA,NaN,NaN,13,,13,FILE 54602,,LOS ANGELES,CA,900749998,,NovaNet - MCMC - RHP
1,BE9124A610A9998tksft,CLU20241115325204,1,351166081,JOHNSON MEMORIAL HOSPITAL,1125 W JEFFERSON STREET,FRANKLIN,IN,461312140,QWCR14517,FISHER TERRI,1/11/1974,100600,NaN,M7672,M722,M7732,NaN,10/17/2024,10/17/2024,510,G0463,NaN,NaN,1,65.0,65.00,N,282N00000X,IN,NaN,1.578930e+09,13,,13,1125 W JEFFERSON STREET,,FRANKLIN,IN,461312140,,Lucent - RHP


In [7]:
Provider_TIN_dict = {'TIN':['Name', 'Unique_TIN__c'], 'Address':['Address__c'], 'City':['City__c'], 'State':['State__c'], 'Zip':['Zip__c']}

new_columns = [col for mapped_cols in Provider_TIN_dict.values() for col in mapped_cols]
new_df = pd.DataFrame(columns=new_columns)

# Populate the new DataFrame by mapping values
for source_col, target_cols in Provider_TIN_dict.items():
    if source_col in Source_data.columns:
        for target_col in target_cols:
            new_df[target_col] = Source_data[source_col]

# Display the new DataFrame
new_df.head()






,Name,Unique_TIN__c,Address__c,City__c,State__c,Zip__c
0,941105628,941105628,FILE 54602,LOS ANGELES,CA,900749998
1,351166081,351166081,1125 W JEFFERSON STREET,FRANKLIN,IN,461312140
2,610461753,610461753,PO BOX 638704,CINCINNATI,OH,452638704
3,610461753,610461753,PO BOX 638704,CINCINNATI,OH,452638704
4,610461753,610461753,PO BOX 638704,CINCINNATI,OH,452638704


In [62]:
table_store['Provider_TIN__c'].loc[:10, ['Name','Unique_TIN__c','Address__c','City__c','State__c','Zip__c']]
# table_store['Provider_TIN__c'].head()

,Name,Unique_TIN__c,Address__c,City__c,State__c,Zip__c
0,000000002,000000002,PO Box 362,Royal Oak,MI,480680000
1,000000009,000000009,[Replaced Address] 12288,COLUMBUS,GA,319179998
2,00000002,00000002,None,None,None,None
3,000000021,000000021,[Replaced Address] 12288,COLUMBUS,GA,319179998
4,000000022,000000022,[Replaced Address] 12288,COLUMBUS,GA,319179998
5,003361990,003361990,124 OCEAN AVE,PORTLAND,ME,4103
6,003601492,003601492,MANCHESTER PLAZA,MANCHESTER,NJ,087595804
7,008343062,008343062,124 E MT PLEASANT AVE,LIVINGSTON,NJ,070393026
8,008365900,008365900,PO BOX 3488 DEPT 05090,TUPELO,MS,38803
9,010211501,010211501,PO BOX 21421,NEW YORK CITY,NY,100870000


In [12]:

###### DF as input containing mapped data ready for upsert onto salesforce obj



for i in range(len(new_df)):
    print(i)
    row = new_df.iloc[i, :].to_dict()
    upsert_dict = {k: v for k, v in row.items() if k != 'Unique_TIN__c'}
    print(row)
    # print(upsert_dict)
    try:
        
        sf.Provider_TIN__c.upsert(f"Unique_TIN__c/{row['Unique_TIN__c']}", upsert_dict)
    except Exception as e:
        print(f"Error during upsert: {e}")


    break


### upsert syntax: sf.Provider_TIN__c.upsert(f"Unique_TIN__c/{tax_id}", provider_tin_data); provider_tin_data is data dictionary 


0
{'Name': 941105628, 'Unique_TIN__c': 941105628, 'Address__c': 'FILE 54602', 'City__c': 'LOS ANGELES', 'State__c': 'CA', 'Zip__c': 900749998}


In [21]:
providers_data_pull = table_store['Provider_TIN__c'].loc[:, ['Name','Unique_TIN__c','Address__c','City__c','State__c','Zip__c']]
table_store['Provider_TIN__c'][table_store['Provider_TIN__c']['Unique_TIN__c'] == '941105628']



,Id,OwnerId,IsDeleted,Name,CreatedDate,CreatedById,LastModifiedDate,LastModifiedById,SystemModstamp,LastActivityDate,LastViewedDate,LastReferencedDate,Special_Repricing__c,Contact__c,Phone__c,Fax__c,Email__c,Notes__c,Unique_TIN__c,Address__c,City__c,State__c,Zip__c,Shelby_Co_Contract__c,City_of_Covington_Contract__c,Provider_Name__c,Avg_WC_Savings__c
7228,a0F4W00000W1Ll7UAF,0054W00000ASrc5QAD,False,941105628,2020-09-06T02:00:56.000+0000,0054W00000ASrc5QAD,2024-12-10T19:11:39.000+0000,005VF00000AhMNjYAN,2024-12-10T19:11:39.000+0000,None,2024-12-10T19:11:39.000+0000,2024-12-10T19:11:39.000+0000,False,None,None,None,None,None,941105628,FILE 54602,LOS ANGELES,CA,900749998,None,None,KAISER FOUNDATION HOS OAK,68.466933


In [24]:
sorted(table_store['Account'].columns)



['AccountSource',
 'Account_Acronym__c',
 'AnnualRevenue',
 'BillingAddress',
 'BillingCity',
 'BillingCountry',
 'BillingCountryCode',
 'BillingGeocodeAccuracy',
 'BillingLatitude',
 'BillingLongitude',
 'BillingPostalCode',
 'BillingState',
 'BillingStateCode',
 'BillingStreet',
 'Billing_Address__City__s',
 'Billing_Address__CountryCode__s',
 'Billing_Address__GeocodeAccuracy__s',
 'Billing_Address__Latitude__s',
 'Billing_Address__Longitude__s',
 'Billing_Address__PostalCode__s',
 'Billing_Address__StateCode__s',
 'Billing_Address__Street__s',
 'Billing_Address__c',
 'Claims_System__c',
 'Copy_Billing_Address_to_Shipping_Address__c',
 'CreatedById',
 'CreatedDate',
 'Default_Pricing_Method__c',
 'Description',
 'Email__c',
 'External_ID__c',
 'Fax',
 'Id',
 'Industry',
 'Insurance_Type__c',
 'Integration_Setting_Name__c',
 'IsCustomerPortal',
 'IsDeleted',
 'IsPartner',
 'Jigsaw',
 'JigsawCompanyId',
 'LastActivityDate',
 'LastModifiedById',
 'LastModifiedDate',
 'LastReferencedDat

In [ ]:


table_store['Jurisdiction__c'].loc[:, ['Id', 'Name']]


,Id,Name
0,a024W00000HYAj6QAH,AK
1,a024W00000HYAj7QAH,AL
2,a024W00000HYAj8QAH,AR
3,a024W00000HYAj9QAH,AZ
4,a024W00000HYAjAQAX,CO
5,a024W00000HYAjBQAX,DC
6,a024W00000HYAjCQAX,DE
7,a024W00000HYAjDQAX,FL
8,a024W00000HYAjEQAX,GA
9,a024W00000HYAjFQAX,HI


In [67]:
import numpy as np
map_table = 'DRG__c'
pulled_data = table_store[map_table].loc[:, ['Id', 'Name']]
# pulled_data[pulled_data['Group_Number__c'].isin(list(Source_data['Patient Group/Policy Number']))]

string_ids = [str(ids) for ids in Source_data['DRG']]
Source_data['DRG'] = string_ids


merged = Source_data.merge(table_store[map_table].loc[:, ['Id', 'Name']], left_on='DRG', right_on='Name', how='left')
merged['DRGCode_id'] = merged['Id']
# for target_col in target_cols:
#     new_df[target_col] = merged['Id']

In [59]:
temp = table_store[map_table].loc[0:0, ['Id', 'Name']].values
temp[0][1]

'224'

In [64]:
temp = table_store[map_table].loc[:, ['Id', 'Name']]

temp[temp['Name'].isin(['232', '794', '856', '329', '627'])]

,Id,Name
9,a014W00000ym0QjQAI,232
94,a014W00000ym0S6QAI,329
358,a014W00000ym0WMQAY,627
501,a014W00000ym0YfQAI,794
561,a014W00000ym0ZdQAI,856


In [68]:
merged

,Claim ID,Claim Number,Line ID,TIN,Provider,Address,City,State,Zip,Patient ID,Patient,Date of Birth,Patient Group/Policy Number,DRG,Diag Code 1st,Diag Code 2nd,Diag Code 3rd,Diag Code 4th,Begin DOS,End DOS,Revenue Code,HCPCS/CPT Code,Mod,Mod (2nd),Units,Billed Amount,MAR,IsQPA,Billing Provider Taxonomy,JurisdictionState,4LCodes,NPI,PlaceofService,AnesthesiaUnits,Billtype,BillProvAddress1,BillProvAddress2,BillProvCity,BillProvState,BillProvZip,DRGCode,ClientName,Id,Name,DRGCode_id
0,A3038B5E4B6C914tksft,4.32E+14,1,941105628,KAISER FOUNDATION HOSPITALS,FILE 54602,LOS ANGELES,CA,nan,569713835,WATKINS MONIC,8/12/1982,NNMCMC,nan,M25562,NaN,NaN,NaN,9/26/2024,9/26/2024,320,73564,NaN,NaN,1,987.00,156.72,N,282N00000X,CA,NaN,NaN,13,,13,FILE 54602,,LOS ANGELES,CA,900749998,,NovaNet - MCMC - RHP,NaN,NaN,NaN
1,BE9124A610A9998tksft,CLU20241115325204,1,351166081,JOHNSON MEMORIAL HOSPITAL,1125 W JEFFERSON STREET,FRANKLIN,IN,nan,QWCR14517,FISHER TERRI,1/11/1974,100600,nan,M7672,M722,M7732,NaN,10/17/2024,10/17/2024,510,G0463,NaN,NaN,1,65.00,65.00,N,282N00000X,IN,NaN,1.578930e+09,13,,13,1125 W JEFFERSON STREET,,FRANKLIN,IN,461312140,,Lucent - RHP,NaN,NaN,NaN
2,317D671330C4442tksft,U424319281216,1,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,nan,FBDS11454,COBB BETHAN,3/12/1984,100660,nan,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,278,C1889,NaN,NaN,1,3.00,3.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP,NaN,NaN,NaN
3,317D671330C4442tksft,U424319281216,2,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,nan,FBDS11454,COBB BETHAN,3/12/1984,100660,nan,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,300,81025,NaN,NaN,1,54.00,54.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP,NaN,NaN,NaN
4,317D671330C4442tksft,U424319281216,3,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,nan,FBDS11454,COBB BETHAN,3/12/1984,100660,nan,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,310,88305,59,NaN,4,1000.00,1000.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP,NaN,NaN,NaN
5,317D671330C4442tksft,U424319281216,4,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,nan,FBDS11454,COBB BETHAN,3/12/1984,100660,nan,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,310,88342,NaN,NaN,1,1159.00,1159.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP,NaN,NaN,NaN
6,317D671330C4442tksft,U424319281216,5,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,nan,FBDS11454,COBB BETHAN,3/12/1984,100660,nan,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,370,NaN,NaN,NaN,3,1227.00,1227.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP,NaN,NaN,NaN
7,317D671330C4442tksft,U424319281216,6,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,nan,FBDS11454,COBB BETHAN,3/12/1984,100660,nan,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,636,J2704,NaN,NaN,70,155.00,155.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP,NaN,NaN,NaN
8,317D671330C4442tksft,U424319281216,7,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,nan,FBDS11454,COBB BETHAN,3/12/1984,100660,nan,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,710,NaN,NaN,NaN,3,48.00,48.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP,NaN,NaN,NaN
9,317D671330C4442tksft,U424319281216,8,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,nan,FBDS11454,COBB BETHAN,3/12/1984,100660,nan,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,750,43239,NaN,NaN,1,3326.00,3326.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP,NaN,NaN,NaN
